In [5]:
#!/usr/bin/env python3
import numpy as np
from ikpy.chain import Chain
from ikpy.link import OriginLink, URDFLink
from dynamixel_sdk import PortHandler, PacketHandler

# ----------------------------
# Define the Kinematic Chain (in meters)
# ----------------------------
# Joint 7: Base rotation (rotates about z)
joint7 = URDFLink(
    name="7_base_rotation",
    origin_translation=np.array([0, 0, 0.038]),  # 38 mm upward from base
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 0, 1]),
    joint_type="revolute",
    bounds=(0, 3*np.pi/2)
)

# Joint 8: Shoulder tilt (rotates about y, here using [0, -1, 0])
joint8 = URDFLink(
    name="8_shoulder_tilt",
    origin_translation=np.array([0, 0, 0.017]),  # 17 mm upward from Joint 7
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, -1, 0]),
    joint_type="revolute",
    bounds=(-.65, np.pi/2)
)

# Joint 9: Elbow tilt (rotates about y)
L2 = 0.110  # 110 mm from Joint 8
joint9 = URDFLink(
    name="9_elbow_tilt",
    origin_translation=np.array([0, 0, L2]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, -1, 0]),
    joint_type="revolute",
    bounds=(-110 / 180 * np.pi, 1.4)
)

# Joint 10: Wrist tilt (rotates about y)
L3 = 0.100  # 100 mm from Joint 9
joint10 = URDFLink(
    name="10_wrist_tilt",
    origin_translation=np.array([-L3, 0, 0]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 1, 0]),
    joint_type="revolute",
    bounds=(-np.pi/2, np.pi/2)
)

# Joint 11: Gripper rotate (rotates about z)
L4 = 0.120 # 45 mm from Joint 10
joint11 = URDFLink(
    name="11_gripper_rotate",
    origin_translation=np.array([-L4, 0, 0]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([1, 0, 0]),
    joint_type="revolute"
)

# Assemble the robot chain: The first link is the fixed base (OriginLink)
robot_chain_11 = Chain(name="my_robot_arm_upto_joint11", links=[
    OriginLink(),
    joint7,
    joint8,
    joint9,
    joint10,
    joint11
])

# ----------------------------
# Dynamixel Configuration
# ----------------------------
PROTOCOL_VERSION = 2.0
DXL_IDS = [7, 8, 9, 10, 11, 12]  # Use first 5 IDs for joints 7-11.
BAUDRATE = 1e6
DEVICENAME = '/dev/tty.usbmodem58FD0170871'  # Replace with your serial port

TICKS_PER_REV = 4096

def rad_to_dxl(angle_rad):
    angle_rad = angle_rad
    return int((angle_rad / (2 * np.pi)) * TICKS_PER_REV)

def dxl_to_rad(ticks):
    return (ticks / TICKS_PER_REV) * 2 * np.pi

# ----------------------------
# Initialize Port and Packet Handler (No torque enable required here)
# ----------------------------
port_handler = PortHandler(DEVICENAME)
packet_handler = PacketHandler(PROTOCOL_VERSION)
if not port_handler.openPort():
    print("Failed to open port!")
    exit()
if not port_handler.setBaudRate(BAUDRATE):
    print("Failed to set baudrate!")
    exit()

def to_signed(val):
    if val > 0x7FFFFFFF:  # 0x7FFFFFFF is 2^31 - 1
        return val - 0x100000000  # 0x100000000 is 2^32
    return val

# ----------------------------
# Read Current Angles Function
# ----------------------------
def read_current_angles():
    angles = []
    for dxl_id in DXL_IDS[:5]:  # Only for joints 7-11
        pos, dxl_comm_result, dxl_error = packet_handler.read4ByteTxRx(port_handler, dxl_id, 132)
        print("ID", dxl_id, "TICKS", pos)
        if dxl_comm_result != 0 or dxl_error != 0:
            print(f"Error reading servo {dxl_id}, assuming 0.")
            angles.append(0)
        else:
            angle = dxl_to_rad(to_signed(pos))
            print("RAW POS", pos)
            print("SIGNED POS", to_signed(pos))
            angles.append(angle)
    return np.array(angles)

# ----------------------------
# Forward Kinematics Function for Chain Up to Joint 11
# ----------------------------
def forward_kinematics_11(angles):
    truncated = angles[:5]  # angles for joints 7-11
    full_angles = np.concatenate(([0], truncated))  # Prepend dummy for OriginLink
    return robot_chain_11.forward_kinematics(full_angles)

# ----------------------------
# Main Execution: Read current angles, compute FK, and print position (in cm)
# ----------------------------
current_angles = read_current_angles()
print("Current joint angles (deg) from servos:", current_angles*180/ (np.pi))
print("Current joint angles (rad) from servos:", current_angles)

home_pose = forward_kinematics_11(current_angles)
home_xyz = home_pose[:3, 3] * 100  # Convert from meters to centimeters

print("\nHome position (xyz) of end-effector (joint 11) relative to base (in cm):")
print(f"  x = {home_xyz[0]:.2f} cm")
print(f"  y = {home_xyz[1]:.2f} cm")
print(f"  z = {home_xyz[2]:.2f} cm")

port_handler.closePort()


ID 7 TICKS 2042
RAW POS 2042
SIGNED POS 2042
ID 8 TICKS 649
RAW POS 649
SIGNED POS 649
ID 9 TICKS 4294967194
RAW POS 4294967194
SIGNED POS -102
ID 10 TICKS 569
RAW POS 569
SIGNED POS 569
ID 11 TICKS 3887
RAW POS 3887
SIGNED POS 3887
Current joint angles (deg) from servos: [179.47265625  57.04101562  -8.96484375  50.00976562 341.63085938]
Current joint angles (rad) from servos: [ 3.13238877  0.99555353 -0.15646604  0.87283507  5.96258332]

Home position (xyz) of end-effector (joint 11) relative to base (in cm):
  x = 27.90 cm
  y = -0.26 cm
  z = 4.45 cm


/Users/dimayanovsky/code/marvin/venv/lib/python3.12/site-packages/ikpy/chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))


In [6]:
#======MOVE
import numpy as np
import time
from ikpy.chain import Chain
from ikpy.link import OriginLink, URDFLink
from dynamixel_sdk import PortHandler, PacketHandler

# ----------------------------
# Initialize Port and Packet Handler, and enable torque
# ----------------------------
port_handler = PortHandler(DEVICENAME)
packet_handler = PacketHandler(PROTOCOL_VERSION)

def to_unsigned(val):
    if val < 0:
        return val + 0x100000000  # 0x100000000 is 2^32
    return val


if not port_handler.openPort():
    print("Failed to open port!")
    exit()
if not port_handler.setBaudRate(BAUDRATE):
    print("Failed to set baudrate!")
    exit()

# Enable torque on servos for joints 7-11 (first 5 IDs)
TORQUE_ENABLE_ADDR = 64  # Typical address for torque enable in protocol 2.0
TORQUE_ENABLE_VALUE = 1
for dxl_id in DXL_IDS[:5]:
    comm_result, error = packet_handler.write1ByteTxRx(port_handler, dxl_id, TORQUE_ENABLE_ADDR, TORQUE_ENABLE_VALUE)
    if comm_result != 0 or error != 0:
        print(f"Error enabling torque for servo {dxl_id}: {comm_result}, {error}")
    else:
        print(f"Torque enabled for servo {dxl_id}")

# ----------------------------
# Function to Read Current Angles (radians) from Servos (joints 7-11)
# ----------------------------
def read_current_angles():
    angles = []
    for dxl_id in DXL_IDS[:5]:
        pos, comm_result, error = packet_handler.read4ByteTxRx(port_handler, dxl_id, 132)
        if comm_result != 0 or error != 0:
            print(f"Error reading servo {dxl_id}, assuming 0.")
            angles.append(0)
        else:
            angles.append(dxl_to_rad(to_signed(pos)))
    return np.array(angles)

# ----------------------------
# Forward Kinematics Function for Chain Up to Joint 11
# ----------------------------
def forward_kinematics_11(angles):
    """
    Computes FK for joints 7-11.
    Prepend a dummy value (0) for the base (OriginLink) and compute the 4x4 pose.
    """
    full_angles = np.concatenate(([0], angles))
    return robot_chain_11.forward_kinematics(full_angles)

# ----------------------------
# Build a Target Pose (in meters)
# ----------------------------
def build_target_pose(x, y, z):
    pose = np.eye(4)
    pose[:3, 3] = np.array([x, y, z])
    return pose

# ----------------------------
# Move to a Target Based on Current Position (using current configuration as initial guess)
# ----------------------------
def move_to_target_from_current(target_x_cm, target_y_cm, target_z_cm, current_angles=None):
    # Read current servo angles (radians) for joints 7-11.
    if current_angles is None:
        current_angles = read_current_angles()
    print("Current joint angles (radians):", current_angles * 180/np.pi)
    
    # Full configuration: prepend 0 for the base.
    current_full_angles = np.concatenate(([0], current_angles))
    
    # Build target pose from user input (convert cm to m).
    target_pose = build_target_pose(target_x_cm / 100.0, target_y_cm / 100.0, target_z_cm / 100.0)
    print("Target pose matrix (meters):")
    print(target_pose)
    
    # Compute IK using current configuration as the initial guess.
    ik_solution = robot_chain_11.inverse_kinematics_frame(target_pose, initial_position=current_full_angles)
    # Extract joint angles for joints 7-11 (ignore the dummy base element at index 0).
    desired_angles = ik_solution[1:6]
    
    print("\nCalculated joint angles (radians) for target position:")
    # Print the angles directly in radians.
    # for i, angle in enumerate(desired_angles):
    #     print(f"  Joint {7+i}: {angle*180/np.pi} degree")
    
    # Command servos (for joints 7-11) using the computed angles.
    for i, dxl_id in enumerate(DXL_IDS[:5]):
        target_tick = to_unsigned(rad_to_dxl(desired_angles[i]))
        print("WRITING TICKS SIGNED: ", rad_to_dxl(desired_angles[i]))
        print("TICKS UNSIGNED", target_tick)
        
        comm_result, error = packet_handler.write4ByteTxRx(port_handler, dxl_id, 116, target_tick)
        if comm_result != 0 or error != 0:
            print(f"Error sending command to servo {dxl_id}: {comm_result}, {error}")
        else:
            print(f"Servo {dxl_id} commanded to {desired_angles[i]:.3f} rad ({target_tick} ticks)")
    
    # Read back new servo angles and compute the new end-effector pose.
    # new_angles = read_current_angles()
    # new_pose = forward_kinematics_11(new_angles)
    # new_xyz = new_pose[:3, 3] * 100  # convert from m to cm
    # print("\nNew end-effector position (cm):")
    # print(f"  x = {new_xyz[0]:.2f} cm, y = {new_xyz[1]:.2f} cm, z = {new_xyz[2]:.2f} cm")

# ----------------------------
# Main Interactive Loop
# ----------------------------
# def main_loop():
#     while True:
#         current_angles = read_current_angles()
#         current_pose = forward_kinematics_11(current_angles)
#         current_xyz = current_pose[:3, 3] * 100  # in cm
#         print("\nCurrent end-effector position (cm):")
#         print(f"  x = {current_xyz[0]:.2f} cm, y = {current_xyz[1]:.2f} cm, z = {current_xyz[2]:.2f} cm")
        
#         user_input = input("\nEnter new target coordinates (x y z in cm), or 'q' to quit: ")
#         if user_input.strip().lower() == 'q':
#             break
#         try:
#             tx, ty, tz = map(float, user_input.strip().split())
#         except Exception:
#             print("Invalid input. Please enter three numbers separated by spaces.")
#             continue
        
#         move_to_target_from_current(tx, ty, tz)
    
#     port_handler.closePort()
#     print("Port closed. Exiting.")

# if __name__ == '__main__':
#     main_loop()


Torque enabled for servo 7
Torque enabled for servo 8
Torque enabled for servo 9
Torque enabled for servo 10
Torque enabled for servo 11


In [3]:
import torch
from SingleSessionSingleTrialDataset import SingleSessionSingleTrialDataset
import numpy as np
from pynwb import NWBHDF5IO

import os

dataset_path = "000070"
nwb_file_path = os.path.join(
    dataset_path, "sub-Jenkins", "sub-Jenkins_ses-20090916_behavior+ecephys.nwb")
nwb_file_path = "sub-Jenkins_ses-20090916_behavior+ecephys.nwb"
io = NWBHDF5IO(nwb_file_path, 'r')
nwb_file = io.read()
hand_data = nwb_file.processing['behavior'].data_interfaces['Position']['Hand'].data[:]
hand_timestamps = nwb_file.processing['behavior'].data_interfaces['Position']['Hand'].timestamps[:]
trial_data = nwb_file.intervals['trials']

unit_spike_times = [nwb_file.units[unit_id]['spike_times'].iloc[0][:]
                    for unit_id in range(len(nwb_file.units))]
n_neurons = len(unit_spike_times)
n_context_bins = 50

trials_start_from = int(2000 * 0.9)
n_trials = int(2000 * 0.1)
datasets = [SingleSessionSingleTrialDataset(
    trial_data, hand_data, hand_timestamps, unit_spike_times, trial_id, bin_size=0.02, n_context_bins=n_context_bins) for trial_id in range(trials_start_from, trials_start_from + n_trials)]
dataset = torch.utils.data.ConcatDataset(datasets)
print(f"Dataset from {n_trials} trials has {len(dataset)} samples")

/Users/dimayanovsky/code/marvin/venv/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.3.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/dimayanovsky/code/marvin/venv/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.5 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Dataset from 200 trials has 20418 samples


In [4]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert dataset to PyTorch tensors and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = []
y = []
for i in range(len(dataset)):
    features, labels = dataset[i]
    X.append(features[:].flatten())
    y.append(labels)
X_test = torch.stack(X).to(device)
y_test = torch.stack(y).to(device)

# Define model
input_size = n_neurons * n_context_bins
model = nn.Sequential(
    nn.Linear(input_size, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
).to(device)

# Load the trained model
model.load_state_dict(torch.load('trained_mlp_model.pth'))
model.eval()  # Set the model to evaluation mode


Sequential(
  (0): Linear(in_features=9600, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
import pygame
import numpy as np
import time

# Initialize Pygame
pygame.init()

# # Set up display
# screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
# WIDTH, HEIGHT = screen.get_size()
# Set up display
WIDTH = 1200
HEIGHT = 800  # Increased height to accommodate velocity plots
screen = pygame.display.set_mode((WIDTH, HEIGHT))

pygame.display.set_caption("Neural Spike Train and Velocity Visualization")

# Colors
BLACK = (0, 0, 0)
GRAY = (140, 140, 140)  # For grid lines
WHITE = (255, 255, 255)
DARK_GRAY = (40, 40, 40)  # Darker gray for velocity lines

# X offset for plots
X_OFFSET = 45

# Get spike data
spike_data = X_test.reshape(-1, n_neurons, n_context_bins)[:, :, -1].T
total_bins = spike_data.shape[1]
window_size = 500
bin_step = 1  # Number of bins to advance each frame (1 bin = 20ms)

# Get model predictions
y_pred = model(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()
y_true = y_test.detach().numpy()

# Normalize predictions and true values to have mean 1
y_pred = y_pred / np.mean(np.abs(y_pred))
y_true = y_true / np.mean(np.abs(y_true))

# Calculate scaling factors
spike_plot_height = HEIGHT // 4 * 3
neuron_height = spike_plot_height // n_neurons
time_bin_width = WIDTH // window_size
plot_height = HEIGHT // 10  # Reduced height for each velocity plot

# Normalize spike data for color intensity
spike_data_normalized = (spike_data - spike_data.min()) / (spike_data.max() - spike_data.min())
spike_data_normalized = spike_data_normalized.detach().numpy()

# Create font for labels
font = pygame.font.SysFont('arial', 24)

def normalize_for_plot(value, height):
    # Normalize values to fit in plot height
    return height // 2 + (value * height // 28)

running = True
current_bin = 0
clock = pygame.time.Clock()

# Pre-create surface for spike data
spike_surface = pygame.Surface((WIDTH, spike_plot_height))

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:  # Add escape key to exit
                running = False
            
    # Clear screen
    screen.fill(BLACK)
    spike_surface.fill(BLACK)
    
    # Draw spike trains first using numpy operations
    if current_bin + window_size <= total_bins:
        # Get the current window of spike data
        window_data = spike_data_normalized[:, current_bin:current_bin+window_size]
        
        # Convert to pixel values (0-255)
        pixel_values = np.minimum(window_data * 255 * 1.5, 255).astype(np.uint8)
        
        # Create a surface from the numpy array
        for neuron in range(n_neurons):
            row_data = pixel_values[neuron]
            for t, intensity in enumerate(row_data):
                if intensity > 0:  # Only draw if there's activity
                    pygame.draw.rect(spike_surface, (intensity, intensity, intensity),
                                   (X_OFFSET + t * time_bin_width, neuron * neuron_height,
                                    time_bin_width, neuron_height))
    
    # Draw the spike surface to the screen
    screen.blit(spike_surface, (0, 0))
    
    # Draw grid lines and channel numbers on top
    for i in range(0, spike_plot_height, neuron_height * n_neurons):  # Draw every 200 channels
        pygame.draw.line(screen, GRAY, (X_OFFSET, i), (WIDTH, i), 1)
        # Draw channel number
        label = font.render(str(i // neuron_height), True, WHITE)
        # Rotate the label surface
        rotated_label = pygame.transform.rotate(label, 90)
        screen.blit(rotated_label, (10, i))
    
    # Draw velocity plots
    y_offset = spike_plot_height - 80  # Start below spike plot
    
    # Draw X velocity plot
    pygame.draw.line(screen, DARK_GRAY, (X_OFFSET, y_offset + plot_height//2), (WIDTH, y_offset + plot_height//2), 1)
    
    # Pre-calculate positions for velocity plots
    if current_bin + window_size <= len(y_true):
        t_range = np.arange(window_size-1)
        x_coords = X_OFFSET + t_range * time_bin_width
        x_coords_next = X_OFFSET + (t_range + 1) * time_bin_width
        
        # X velocity
        true_y_x = y_offset + normalize_for_plot(y_true[current_bin:current_bin+window_size-1, 0], plot_height)
        true_y_x_next = y_offset + normalize_for_plot(y_true[current_bin+1:current_bin+window_size, 0], plot_height)
        pred_y_x = y_offset + normalize_for_plot(y_pred[current_bin:current_bin+window_size-1, 0], plot_height)
        pred_y_x_next = y_offset + normalize_for_plot(y_pred[current_bin+1:current_bin+window_size, 0], plot_height)
        
        # Draw lines in batches
        for i in range(len(x_coords)):
            pygame.draw.line(screen, GRAY, 
                           (int(x_coords[i]), int(true_y_x[i])), 
                           (int(x_coords_next[i]), int(true_y_x_next[i])), 2)
            pygame.draw.line(screen, WHITE,
                           (int(x_coords[i]), int(pred_y_x[i])),
                           (int(x_coords_next[i]), int(pred_y_x_next[i])), 2)
    
    # Draw Y velocity plot
    y_offset += plot_height + 110
    pygame.draw.line(screen, DARK_GRAY, (X_OFFSET, y_offset + plot_height//2), (WIDTH, y_offset + plot_height//2), 1)
    
    if current_bin + window_size <= len(y_true):
        # Y velocity
        true_y_y = y_offset + normalize_for_plot(y_true[current_bin:current_bin+window_size-1, 1], plot_height)
        true_y_y_next = y_offset + normalize_for_plot(y_true[current_bin+1:current_bin+window_size, 1], plot_height)
        pred_y_y = y_offset + normalize_for_plot(y_pred[current_bin:current_bin+window_size-1, 1], plot_height)
        pred_y_y_next = y_offset + normalize_for_plot(y_pred[current_bin+1:current_bin+window_size, 1], plot_height)
        
        # Draw lines in batches
        for i in range(len(x_coords)):
            pygame.draw.line(screen, GRAY,
                           (int(x_coords[i]), int(true_y_y[i])),
                           (int(x_coords_next[i]), int(true_y_y_next[i])), 2)
            pygame.draw.line(screen, WHITE,
                           (int(x_coords[i]), int(pred_y_y[i])),
                           (int(x_coords_next[i]), int(pred_y_y_next[i])), 2)
    
    # Draw axis labels
    time_label = font.render("Time", True, WHITE)
    channels_label = font.render("Channels", True, WHITE)
    x_vel_label = font.render("velocity X (prediction: WHITE)", True, WHITE)
    y_vel_label = font.render("velocity Y (prediction: WHITE)", True, WHITE)
    
    screen.blit(time_label, (WIDTH // 2 - 30, HEIGHT - 30))
    screen.blit(x_vel_label, (WIDTH // 2 - 150, spike_plot_height - 90))
    screen.blit(y_vel_label, (WIDTH // 2 - 150, spike_plot_height + plot_height - 15))
    
    # Rotate and draw y-axis label
    channels_surface = pygame.Surface((200, 30))
    channels_surface.fill(BLACK)
    channels_surface.blit(channels_label, (50, 0))
    channels_surface = pygame.transform.rotate(channels_surface, 90)
    screen.blit(channels_surface, (10, spike_plot_height // 2 - 100))
    
    # Update display
    pygame.display.flip()
    
    # Move window by one bin (20ms) each frame
    current_bin += bin_step
    if current_bin + window_size > total_bins:
        current_bin = 0

    lambda_decay = 0.99  # Exponential decay factor
    integrated_v = np.zeros(2)
    for i in range(current_bin + window_size):
        integrated_v = lambda_decay * integrated_v + y_pred[i]
    integrated_v = integrated_v / 70
    integrated_v = integrated_v.clip(-1, 1)
    integrated_v = integrated_v * 10
    print(integrated_v)
    # now integrated v is always in the window -10cm to 10cm
    x, y = integrated_v[0], integrated_v[1]

    current_angles = read_current_angles()
    # current_pose = forward_kinematics_11(current_angles)
    # current_xyz = current_pose[:3, 3] * 100  # in cm
    # print("\nCurrent end-effector position (cm):")
    # print(f"  x = {current_xyz[0]:.2f} cm, y = {current_xyz[1]:.2f} cm, z = {current_xyz[2]:.2f} cm")

    move_to_target_from_current(15, x, y+25, current_angles=current_angles)
        
    # Control frame rate to 50 FPS (20ms per frame)
    clock.tick(50)

pygame.quit()


/var/folders/w8/9p7v69054xq825yv1b7wh2440000gn/T/ipykernel_70981/70834090.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = model(torch.tensor(X_test, dtype=torch.float32)).detach().numpy()


[ 10. -10.]
Current joint angles (radians): [194.67773438 -29.8828125   36.9140625   60.29296875 341.63085938]
Target pose matrix (meters):
[[1.   0.   0.   0.1 ]
 [0.   1.   0.   0.1 ]
 [0.   0.   1.   0.15]
 [0.   0.   0.   1.  ]]

Calculated joint angles (radians) for target position:
WRITING TICKS SIGNED:  2559
TICKS UNSIGNED 2559
Servo 7 commanded to 3.927 rad (2559 ticks)
WRITING TICKS SIGNED:  -86
TICKS UNSIGNED 4294967210
Servo 8 commanded to -0.133 rad (4294967210 ticks)
WRITING TICKS SIGNED:  714
TICKS UNSIGNED 714
Servo 9 commanded to 1.095 rad (714 ticks)
WRITING TICKS SIGNED:  1020
TICKS UNSIGNED 1020
Servo 10 commanded to 1.566 rad (1020 ticks)
WRITING TICKS SIGNED:  3887
TICKS UNSIGNED 3887
Servo 11 commanded to 5.963 rad (3887 ticks)
[ 10. -10.]
Current joint angles (radians): [198.6328125  -26.27929688  39.99023438  64.51171875 341.54296875]
Target pose matrix (meters):
[[1.   0.   0.   0.1 ]
 [0.   1.   0.   0.1 ]
 [0.   0.   1.   0.15]
 [0.   0.   0.   1.  ]]

Calcul

ValueError: Initial guess is outside of provided bounds